In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit.library import RealAmplitudes, TwoLocal
from qiskit.quantum_info import Statevector, Operator, SparsePauliOp, state_fidelity
# No pauli_decompose as it was not found in the user's qiskit-terra

# Import SciPy's optimizer
try:
    from scipy.optimize import minimize
except ModuleNotFoundError:
    print("ERROR: scipy is not installed. Please install it: pip install scipy")
    def minimize(*args, **kwargs): # type: ignore
        raise RuntimeError("scipy.optimize.minimize not found. Please install scipy.")

# --- Configuration ---
NUM_QUBITS = 5
PSI_ANSATZ_DEPTH = 5
PROJECTOR_U_REPS = 1
SCIPY_COBYLA_MAXITER = 100000
SCIPY_COBYLA_TOL = 1e-8
SEED = 100 # Seed for NumPy

# Set NumPy's random seed for reproducibility of NumPy operations
np.random.seed(SEED)
print(f"NumPy random seed set to: {SEED}")

print(f"--- Quantum State and Projector Optimization (Using SciPy COBYLA, No Qiskit Global Seed) ---")
print(f"Configuration: Qubits={NUM_QUBITS}, Psi Depth={PSI_ANSATZ_DEPTH}")
print(f"Optimizer: SciPy COBYLA, Maxiter={SCIPY_COBYLA_MAXITER}, Tol={SCIPY_COBYLA_TOL}\n")

# --- Step 1: Create a random 5-qubit quantum state `psi` ---
print("Step 1: Creating initial state |psi>...")
# The parameters for RealAmplitudes might have an internal seed mechanism
# not controlled by algorithm_globals in older Qiskits, or might rely on numpy's seed.
psi_ansatz = RealAmplitudes(NUM_QUBITS, reps=PSI_ANSATZ_DEPTH, entanglement='linear')
num_psi_params = psi_ansatz.num_parameters
# These random parameters *will* be reproducible due to np.random.seed(SEED)
random_psi_params = np.random.rand(num_psi_params) * 2 * np.pi
psi_circuit = psi_ansatz.assign_parameters(random_psi_params)

try:
    psi_vector = Statevector(psi_circuit)
    print(f"|psi> statevector created from RealAmplitudes with {num_psi_params} parameters.")
    print(psi_vector)
except Exception as e:
    print(f"CRITICAL ERROR: Could not create Statevector. Qiskit installation is likely still broken: {e}")
    import sys
    sys.exit(1)


# --- Step 2: Define parametrized projectors P_ij(theta) ---
proj_00_on_qubits_static_pauli_sum = {}
for q1_local_idx in range(NUM_QUBITS -1):
    q2_local_idx = q1_local_idx + 1
    pauli_list_for_proj = [
        ("I" * NUM_QUBITS, 0.25),
        ("".join(['Z' if i == q1_local_idx else 'I' for i in range(NUM_QUBITS)]), 0.25),
        ("".join(['Z' if i == q2_local_idx else 'I' for i in range(NUM_QUBITS)]), 0.25),
        ("".join(['Z' if i == q1_local_idx or i == q2_local_idx else 'I' for i in range(NUM_QUBITS)]), 0.25)
    ]
    reversed_pauli_list = [(p_str[::-1], coeff) for p_str, coeff in pauli_list_for_proj]
    proj_00_on_qubits_static_pauli_sum[(q1_local_idx, q2_local_idx)] = SparsePauliOp.from_list(reversed_pauli_list)


u_ansatz_2q = TwoLocal(2, rotation_blocks=['ry', 'rz'], entanglement_blocks='cx', reps=PROJECTOR_U_REPS, entanglement='linear')
num_u_params = u_ansatz_2q.num_parameters
print(f"\nStep 2: Parametrized projectors P_ij(theta) structure defined.")
print(f"U(theta) for projectors is TwoLocal with {num_u_params} parameters (reps={PROJECTOR_U_REPS}).")


# --- Step 3: Optimize each projector's parameters ---
print("\nStep 3: Optimizing projector parameters...")
qubit_pairs_indices = [(i, i + 1) for i in range(NUM_QUBITS - 1)]
optimized_projector_ops_as_operators = []

for q_pair_idx, (q1, q2) in enumerate(qubit_pairs_indices):
    print(f"  Optimizing P_{q1}{q2}(theta) acting on qubits ({q1}, {q2})...")
    
    proj_00_full_static_spo = proj_00_on_qubits_static_pauli_sum[(q1, q2)]

    def cost_function_projector(params_theta):
        try:
            u_circuit_2q = u_ansatz_2q.assign_parameters(params_theta)
            u_full_qc = QuantumCircuit(NUM_QUBITS)
            u_full_qc.compose(u_circuit_2q, qubits=[q1, q2], inplace=True)
            u_full_op = Operator(u_full_qc)
            
            p_ij_full_op_object = u_full_op @ proj_00_full_static_spo @ u_full_op.adjoint()
            
            expectation_value = np.real(psi_vector.expectation_value(p_ij_full_op_object))
            print(f"  P_{q1}{q2} expectation value: {expectation_value:.6f}", end='\r')
            return expectation_value
        except Exception as e_cost:
            print(f"ERROR in cost_function_projector: {e_cost}. Qiskit part is likely broken.")
            return np.nan

    # These initial parameters *will* be reproducible due to np.random.seed(SEED)
    initial_u_params = np.random.rand(num_u_params) * 2 * np.pi
    
    try:
        opt_result = minimize(fun=cost_function_projector,
                              x0=initial_u_params,
                              method='COBYLA',
                              tol=SCIPY_COBYLA_TOL,
                              options={'maxiter': SCIPY_COBYLA_MAXITER, 'disp': True})
        
        optimized_u_params = opt_result.x
        min_expectation_value = opt_result.fun
        nfev = opt_result.nfev
        success = opt_result.success
        message = opt_result.message

        print(f"    P_{q1}{q2}: Optimized value = {min_expectation_value:.6e}, Evals = {nfev}, Success: {success}, Msg: {message}")

        if not success and not np.isnan(min_expectation_value) :
             print(f"    Warning: SciPy COBYLA optimization did not converge successfully for P_{q1}{q2}. Message: {message}")
        elif np.isnan(min_expectation_value):
            print(f"    ERROR: Optimization for P_{q1}{q2} failed (NaN result), likely due to Qiskit errors in cost function.")
            continue

        u_circuit_2q_opt = u_ansatz_2q.assign_parameters(optimized_u_params)
        u_full_qc_opt = QuantumCircuit(NUM_QUBITS)
        u_full_qc_opt.compose(u_circuit_2q_opt, qubits=[q1, q2], inplace=True)
        u_full_op_opt = Operator(u_full_qc_opt)
        
        optimized_p_ij_op_object = u_full_op_opt @ proj_00_full_static_spo @ u_full_op_opt.adjoint()
        optimized_projector_ops_as_operators.append(optimized_p_ij_op_object)

    except RuntimeError as e_scipy:
        print(f"    ERROR: SciPy COBYLA optimization failed for P_{q1}{q2}: {e_scipy}")
        continue
    except Exception as e_general_opt:
        print(f"    ERROR: General error during optimization or reconstruction for P_{q1}{q2}: {e_general_opt}")
        continue

# --- Step 4: Sum optimized projectors into a Hamiltonian H (as an Operator) ---
print("\nStep 4: Constructing Hamiltonian H = sum(P_ij*) as sum of Operators...")
if not optimized_projector_ops_as_operators:
    print("Warning: No optimized projectors were successfully created. Hamiltonian will be zero Operator.")
    identity_op = Operator(np.eye(2**NUM_QUBITS))
    hamiltonian_h_operator = 0.0 * identity_op
else:
    hamiltonian_h_operator = optimized_projector_ops_as_operators[0]
    for i in range(1, len(optimized_projector_ops_as_operators)):
        hamiltonian_h_operator = hamiltonian_h_operator + optimized_projector_ops_as_operators[i]
print(f"Hamiltonian H (as Operator) constructed.")

# --- Step 5: Compute the exact ground state of H via full diagonalization ---
print("\nStep 5: Computing exact ground state of H (from Operator)...")
h_matrix = hamiltonian_h_operator.data

eigenvalues, eigenvectors = np.linalg.eigh(h_matrix)
ground_state_energy = eigenvalues[0]
ground_state_vector_data = eigenvectors[:, 0]
ground_state_vector = Statevector(ground_state_vector_data)

print(f"  Ground state energy of H: {ground_state_energy:.6f}")

# --- Step 6: Calculate fidelity between the ground state and the initial state psi ---
print("\nStep 6: Calculating fidelity between ground state of H and |psi>...")
try:
    fidelity = state_fidelity(psi_vector, ground_state_vector)
    print(f"\n--- FINAL RESULT ---")
    print(f"Fidelity(|psi>, GroundState(H)): {fidelity:.8f}")

    exp_H_psi = np.real(psi_vector.expectation_value(hamiltonian_h_operator))
    print(f"Expectation <psi|H|psi>: {exp_H_psi:.6f}")

    if ground_state_energy > 1e-3:
        print(f"Note: Ground state energy of H ({ground_state_energy:.6f}) is not close to zero.")
    if abs(exp_H_psi - ground_state_energy) > 1e-3 and fidelity < 0.99:
        print(f"Note: <psi|H|psi> ({exp_H_psi:.6f}) differs from H's ground energy.")
except Exception as e_final:
    print(f"ERROR during final calculations (fidelity/expectation): {e_final}. Qiskit functionality is likely still broken.")

NumPy random seed set to: 100
--- Quantum State and Projector Optimization (Using SciPy COBYLA, No Qiskit Global Seed) ---
Configuration: Qubits=5, Psi Depth=1
Optimizer: SciPy COBYLA, Maxiter=100000, Tol=1e-08

Step 1: Creating initial state |psi>...
|psi> statevector created from RealAmplitudes with 10 parameters.
Statevector([-0.04729201+0.j,  0.07234187+0.j, -0.09453827+0.j,
              0.16572145+0.j,  0.16387494+0.j, -0.2863987 +0.j,
             -0.08616016+0.j,  0.1689045 +0.j, -0.00564296+0.j,
              0.02053609+0.j,  0.12660084+0.j, -0.22788068+0.j,
             -0.21031231+0.j,  0.37099108+0.j,  0.15036605+0.j,
             -0.28823681+0.j,  0.15960626+0.j, -0.26275556+0.j,
              0.1035248 +0.j, -0.17216578+0.j, -0.19374094+0.j,
              0.33322438+0.j,  0.03966285+0.j, -0.08796709+0.j,
              0.0766975 +0.j, -0.12316976+0.j,  0.08559972+0.j,
             -0.14712776+0.j, -0.15287013+0.j,  0.26547866+0.j,
              0.06083084+0.j, -0.12245924+

In [33]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit.library import RealAmplitudes, TwoLocal
from qiskit.quantum_info import Statevector, Operator, SparsePauliOp, state_fidelity
# No pauli_decompose needed for this approach if we stick to Operator math

# Import SciPy's optimizer
try:
    from scipy.optimize import minimize
except ModuleNotFoundError:
    print("ERROR: scipy is not installed. Please install it: pip install scipy")
    def minimize(*args, **kwargs): # type: ignore
        raise RuntimeError("scipy.optimize.minimize not found. Please install scipy.")

# --- Configuration ---
NUM_QUBITS = 5
PSI_ANSATZ_DEPTH = 5 # Adjusted from your output for consistency
PROJECTOR_U_REPS = 1
SCIPY_COBYLA_MAXITER = 10000 # From your output
SCIPY_COBYLA_TOL = 1e-7    # From your output
SEED = 43 # Seed for NumPy

# Set NumPy's random seed for reproducibility of NumPy operations
np.random.seed(SEED)
print(f"NumPy random seed set to: {SEED}")

print(f"--- Quantum State Iterative Projection (Using SciPy COBYLA) ---")
print(f"Configuration: Qubits={NUM_QUBITS}, Psi Depth={PSI_ANSATZ_DEPTH}") # PSI_ANSATZ_DEPTH should match config
print(f"Optimizer: SciPy COBYLA, Maxiter={SCIPY_COBYLA_MAXITER}, Tol={SCIPY_COBYLA_TOL}\n")

# --- Step 1: Create a random 5-qubit quantum state `psi_initial` ---
print("Step 1: Creating initial state |psi_initial>...")
psi_ansatz = RealAmplitudes(NUM_QUBITS, reps=PSI_ANSATZ_DEPTH, entanglement='linear')
num_psi_params = psi_ansatz.num_parameters
random_psi_params = np.random.rand(num_psi_params) * 2 * np.pi
psi_circuit = psi_ansatz.assign_parameters(random_psi_params)

try:
    psi_initial_vector = Statevector(psi_circuit)
    print(f"|psi_initial> statevector created with {num_psi_params} parameters.")
    # print(psi_initial_vector) # Optional
except Exception as e:
    print(f"CRITICAL ERROR: Could not create Statevector. Qiskit installation is likely broken: {e}")
    import sys
    sys.exit(1)


# --- Step 2: Define parametrized projectors P_ij(theta) and U(theta) ansatz ---
proj_00_on_qubits_static_pauli_sum = {}
for q1_local_idx in range(NUM_QUBITS -1):
    q2_local_idx = q1_local_idx + 1
    pauli_list_for_proj = [
        ("I" * NUM_QUBITS, 0.25),
        ("".join(['Z' if i == q1_local_idx else 'I' for i in range(NUM_QUBITS)]), 0.25),
        ("".join(['Z' if i == q2_local_idx else 'I' for i in range(NUM_QUBITS)]), 0.25),
        ("".join(['Z' if i == q1_local_idx or i == q2_local_idx else 'I' for i in range(NUM_QUBITS)]), 0.25)
    ]
    reversed_pauli_list = [(p_str[::-1], coeff) for p_str, coeff in pauli_list_for_proj]
    proj_00_on_qubits_static_pauli_sum[(q1_local_idx, q2_local_idx)] = SparsePauliOp.from_list(reversed_pauli_list)


u_ansatz_2q = TwoLocal(2, rotation_blocks=['ry', 'rz'], entanglement_blocks='cx', reps=PROJECTOR_U_REPS, entanglement='linear')
num_u_params = u_ansatz_2q.num_parameters
print(f"\nStep 2: Parametrized projectors P_ij(theta) structure defined.")
print(f"U(theta) for projectors is TwoLocal with {num_u_params} parameters (reps={PROJECTOR_U_REPS}).")


# --- Step 3: Optimize each projector P_ij(theta) against psi_initial ---
print("\nStep 3: Optimizing projector parameters P_ij* against |psi_initial>...")
qubit_pairs_indices = [(i, i + 1) for i in range(NUM_QUBITS - 1)]
optimized_projectors_P_star = []

for q_pair_idx, (q1, q2) in enumerate(qubit_pairs_indices):
    print(f"  Optimizing P_{q1}{q2}(theta) acting on qubits ({q1}, {q2})...")
    
    proj_00_full_static_spo = proj_00_on_qubits_static_pauli_sum[(q1, q2)]

    def cost_function_projector(params_theta):
        try:
            u_circuit_2q = u_ansatz_2q.assign_parameters(params_theta)
            u_full_qc = QuantumCircuit(NUM_QUBITS)
            u_full_qc.compose(u_circuit_2q, qubits=[q1, q2], inplace=True)
            u_full_op = Operator(u_full_qc)
            p_ij_full_op_object = u_full_op @ proj_00_full_static_spo @ u_full_op.adjoint()
            expectation_value = np.real(psi_initial_vector.expectation_value(p_ij_full_op_object))
            return expectation_value
        except Exception as e_cost:
            print(f"ERROR in cost_function_projector for P_{q1}{q2} with params {params_theta}: {e_cost}.")
            return np.nan

    initial_u_params = np.random.rand(num_u_params) * 2 * np.pi
    
    try:
        opt_result = minimize(fun=cost_function_projector,
                              x0=initial_u_params,
                              method='COBYLA',
                              tol=SCIPY_COBYLA_TOL,
                              options={'maxiter': SCIPY_COBYLA_MAXITER, 'disp': False})
        
        optimized_u_params = opt_result.x
        min_expectation_value = opt_result.fun
        nfev = opt_result.nfev
        success = opt_result.success

        print(f"    P_{q1}{q2}*: Optimized <psi_init|P*|psi_init> = {min_expectation_value:.6e}, Evals = {nfev}, Success: {success}")

        if not success and not np.isnan(min_expectation_value) :
             print(f"    Warning: SciPy COBYLA optimization for P_{q1}{q2}* did not converge successfully.")
        elif np.isnan(min_expectation_value):
            print(f"    ERROR: Optimization for P_{q1}{q2}* failed (NaN result). Skipping this projector.")
            continue

        u_circuit_2q_opt = u_ansatz_2q.assign_parameters(optimized_u_params)
        u_full_qc_opt = QuantumCircuit(NUM_QUBITS)
        u_full_qc_opt.compose(u_circuit_2q_opt, qubits=[q1, q2], inplace=True)
        u_full_op_opt = Operator(u_full_qc_opt)
        
        p_ij_star_op_object = u_full_op_opt @ proj_00_full_static_spo @ u_full_op_opt.adjoint()
        optimized_projectors_P_star.append({'qubits': (q1,q2), 'P_star_op': p_ij_star_op_object})

    except RuntimeError as e_scipy:
        print(f"    ERROR: SciPy COBYLA optimization failed for P_{q1}{q2}*: {e_scipy}")
        continue
    except Exception as e_general_opt:
        print(f"    ERROR: General error during optimization P_{q1}{q2}*: {e_general_opt}")
        continue

# --- Step 4: Iteratively apply (I - P_ij*) and renormalize ---
print("\nStep 4: Iteratively applying (I - P_ij*) projection and renormalizing...")

if not optimized_projectors_P_star:
    print("Warning: No optimized projectors P_ij* were successfully created. Cannot perform iterative projection.")
    final_psi_vector = psi_initial_vector.copy() # No projection applied
else:
    current_psi_vector = psi_initial_vector.copy()
    identity_op = Operator(np.eye(2**NUM_QUBITS, dtype=complex))

    projection_order = [(0,1), (1,2), (2,3), (3,4)]

    for q_idx, (q_target1, q_target2) in enumerate(projection_order):
        p_star_info = next((p_info for p_info in optimized_projectors_P_star if p_info['qubits'] == (q_target1, q_target2)), None)

        if p_star_info is None:
            print(f"  Warning: No optimized P_star found for qubits ({q_target1},{q_target2}). Skipping this projection step.")
            continue
        
        p_ij_star_op = p_star_info['P_star_op']
        print(f"  Applying (I - P_{q_target1}{q_target2}*) to current state...")

        k_ij_op = identity_op - p_ij_star_op
        projected_psi_unnormalized = current_psi_vector.evolve(k_ij_op)
        
        # Robust norm calculation
        # Using .data directly to compute norm, as .equiv() might have its own checks
        vec_data = projected_psi_unnormalized.data
        # norm_sq = np.sum(np.abs(vec_data)**2) # Same as np.linalg.norm(vec_data)**2
        norm_val = np.linalg.norm(vec_data)

        if norm_val < 1e-9: # Threshold for norm being effectively zero
            print(f"    WARNING: State norm after applying (I - P_{q_target1}{q_target2}*) is near zero ({norm_val:.2e}).")
            print(f"    This implies the state was almost entirely in the subspace projected onto by P_{q_target1}{q_target2}*.")
            print(f"    Cannot renormalize meaningfully. Stopping iterative projection.")
            # Create a new Statevector from the (potentially zero) data to ensure correct internal state
            current_psi_vector = Statevector(projected_psi_unnormalized.data, dims=projected_psi_unnormalized.dims())
            break 
        
        # Explicitly create a new Statevector from the normalized data
        normalized_data_arr = projected_psi_unnormalized.data / norm_val
        current_psi_vector = Statevector(normalized_data_arr, dims=projected_psi_unnormalized.dims())
        # Verify norm of the new Statevector object
        new_norm_check = np.linalg.norm(current_psi_vector.data)
        print(f"    State renormalized. New norm after explicit creation: {new_norm_check:.7f}")

    final_psi_vector = current_psi_vector

# --- Step 5: Analyze the Final State ---
print("\nStep 5: Final Projected State Analysis...")

# Explicitly re-normalize final_psi_vector before use, just in case of tiny accumulated errors
# Only if its norm is not already zero
final_norm_val = np.linalg.norm(final_psi_vector.data)
if final_norm_val > 1e-9: # If it's not a zero vector
    print(f"  Normalizing final_psi_vector before analysis (current norm: {final_norm_val:.7f})")
    final_psi_vector = Statevector(final_psi_vector.data / final_norm_val, dims=final_psi_vector.dims())
    print(f"  Norm of final_psi_vector after re-check: {np.linalg.norm(final_psi_vector.data):.7f}")
else:
    print(f"  Final state is a near-zero vector (norm: {final_norm_val:.2e}). Analysis might be limited.")


print("Initial state |psi_initial> (first 4 amplitudes):")
print(psi_initial_vector.data[:4])
print("\nFinal state |psi_final> after iterative projections (first 4 amplitudes):")
print(final_psi_vector.data[:4]) # Print the .data attribute

try:
    # Check if final_psi_vector is valid for fidelity calculation
    if final_norm_val < 1e-9: # If it's essentially a zero vector
        print("\nFidelity calculation skipped as final state is a zero vector.")
        fidelity_final_vs_initial = 0.0 # Or handle as appropriate
    else:
        # Ensure psi_initial_vector is also perfectly normalized if it's used elsewhere
        # Though it should be from Statevector(circuit)
        if abs(np.linalg.norm(psi_initial_vector.data) - 1.0) > 1e-9:
             print("Warning: Initial state vector norm is not 1. Re-normalizing for fidelity.")
             psi_initial_vector = Statevector(psi_initial_vector.data / np.linalg.norm(psi_initial_vector.data))

        fidelity_final_vs_initial = state_fidelity(psi_initial_vector, final_psi_vector)
        print(f"\nFidelity(|psi_initial>, |psi_final>): {fidelity_final_vs_initial:.8f}")

    print("\nExpectation values of P_ij* with respect to |psi_initial> (should be small):")
    for p_info in optimized_projectors_P_star:
        q1,q2 = p_info['qubits']
        exp_val = np.real(psi_initial_vector.expectation_value(p_info['P_star_op']))
        print(f"  <psi_initial| P_{q1}{q2}* |psi_initial>: {exp_val:.6e}")

    print("\nExpectation values of P_ij* with respect to |psi_final> (may not be small):")
    for p_info in optimized_projectors_P_star:
        q1,q2 = p_info['qubits']
        if final_norm_val > 1e-9: # Only compute if final_psi_vector is not zero
            exp_val_final = np.real(final_psi_vector.expectation_value(p_info['P_star_op']))
            print(f"  <psi_final| P_{q1}{q2}* |psi_final>: {exp_val_final:.6e}")
        else:
            print(f"  <psi_final| P_{q1}{q2}* |psi_final>: Not computed (final state is near zero vector)")


except Exception as e_final_analysis:
    print(f"ERROR during final state analysis: {e_final_analysis}")
    # For debugging, print the norm of the final vector just before the error
    if 'final_psi_vector' in locals():
        print(f"DEBUG: Norm of final_psi_vector right before error: {np.linalg.norm(final_psi_vector.data)}")
        print(f"DEBUG: final_psi_vector.is_valid(): {final_psi_vector.is_valid(atol=1e-9) if hasattr(final_psi_vector, 'is_valid') else 'is_valid not available'}")


print("\n--- Iterative Projection Script Finished ---")

NumPy random seed set to: 43
--- Quantum State Iterative Projection (Using SciPy COBYLA) ---
Configuration: Qubits=5, Psi Depth=5
Optimizer: SciPy COBYLA, Maxiter=10000, Tol=1e-07

Step 1: Creating initial state |psi_initial>...
|psi_initial> statevector created with 30 parameters.

Step 2: Parametrized projectors P_ij(theta) structure defined.
U(theta) for projectors is TwoLocal with 8 parameters (reps=1).

Step 3: Optimizing projector parameters P_ij* against |psi_initial>...
  Optimizing P_01(theta) acting on qubits (0, 1)...
    P_01*: Optimized <psi_init|P*|psi_init> = 3.526197e-02, Evals = 903, Success: True
  Optimizing P_12(theta) acting on qubits (1, 2)...
    P_12*: Optimized <psi_init|P*|psi_init> = 7.315292e-02, Evals = 450, Success: True
  Optimizing P_23(theta) acting on qubits (2, 3)...
    P_23*: Optimized <psi_init|P*|psi_init> = 3.648237e-02, Evals = 441, Success: True
  Optimizing P_34(theta) acting on qubits (3, 4)...
    P_34*: Optimized <psi_init|P*|psi_init> = 5.

In [42]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit.library import RealAmplitudes, TwoLocal
from qiskit.quantum_info import Statevector, Operator, SparsePauliOp, state_fidelity
# No pauli_decompose needed for this approach if we stick to Operator math

# Import SciPy's optimizer
try:
    from scipy.optimize import dual_annealing # Changed to dual_annealing
except ModuleNotFoundError:
    print("ERROR: scipy is not installed. Please install it: pip install scipy")
    def dual_annealing(*args, **kwargs): # type: ignore
        raise RuntimeError("scipy.optimize.dual_annealing not found. Please install scipy.")

# --- Configuration ---
NUM_QUBITS = 5
PSI_ANSATZ_DEPTH = 5 # From your last output
PROJECTOR_U_REPS = 1
# SCIPY_COBYLA_MAXITER = 1000 # Old config
# SCIPY_COBYLA_TOL = 1e-7    # Old config

# Dual Annealing specific parameters
DA_MAXITER = 5000      # Max iterations for dual_annealing (adjust as needed)
DA_INITIAL_TEMP = 5230 # Default, can be tuned
DA_VISIT = 2.62        # Default visit parameter
DA_ACCEPT = -5.0       # Default accept parameter
# No explicit tolerance parameter like COBYLA's 'tol', convergence is driven by annealing schedule and maxiter.

SEED = 43 # Seed for NumPy

# Set NumPy's random seed for reproducibility of NumPy operations
np.random.seed(SEED)
print(f"NumPy random seed set to: {SEED}")

print(f"--- Quantum State Iterative Projection (Using SciPy Dual Annealing) ---")
print(f"Configuration: Qubits={NUM_QUBITS}, Psi Depth={PSI_ANSATZ_DEPTH}")
print(f"Optimizer: SciPy DualAnnealing, Maxiter={DA_MAXITER}\n")

# --- Step 1: Create a random 5-qubit quantum state `psi_initial` ---
print("Step 1: Creating initial state |psi_initial>...")
psi_ansatz = RealAmplitudes(NUM_QUBITS, reps=PSI_ANSATZ_DEPTH, entanglement='linear')
num_psi_params = psi_ansatz.num_parameters
random_psi_params = np.random.rand(num_psi_params) * 2 * np.pi
psi_circuit = psi_ansatz.assign_parameters(random_psi_params)

try:
    psi_initial_vector = Statevector(psi_circuit)
    print(f"|psi_initial> statevector created with {num_psi_params} parameters.")
except Exception as e:
    print(f"CRITICAL ERROR: Could not create Statevector. Qiskit installation is likely still broken: {e}")
    import sys
    sys.exit(1)


# --- Step 2: Define parametrized projectors P_ij(theta) and U(theta) ansatz ---
proj_00_on_qubits_static_pauli_sum = {}
for q1_local_idx in range(NUM_QUBITS -1):
    q2_local_idx = q1_local_idx + 1
    pauli_list_for_proj = [
        ("I" * NUM_QUBITS, 0.25),
        ("".join(['Z' if i == q1_local_idx else 'I' for i in range(NUM_QUBITS)]), 0.25),
        ("".join(['Z' if i == q2_local_idx else 'I' for i in range(NUM_QUBITS)]), 0.25),
        ("".join(['Z' if i == q1_local_idx or i == q2_local_idx else 'I' for i in range(NUM_QUBITS)]), 0.25)
    ]
    # Assuming your reversal is correct for your Qiskit version's from_list interpretation
    reversed_pauli_list = [(p_str[::-1], coeff) for p_str, coeff in pauli_list_for_proj]
    proj_00_on_qubits_static_pauli_sum[(q1_local_idx, q2_local_idx)] = SparsePauliOp.from_list(reversed_pauli_list)


u_ansatz_2q = TwoLocal(2, rotation_blocks=['ry', 'rz'], entanglement_blocks='cx', reps=PROJECTOR_U_REPS, entanglement='linear')
num_u_params = u_ansatz_2q.num_parameters
print(f"\nStep 2: Parametrized projectors P_ij(theta) structure defined.")
print(f"U(theta) for projectors is TwoLocal with {num_u_params} parameters (reps={PROJECTOR_U_REPS}).")

# Define bounds for the parameters of U(theta) - typically angles (0 to 2*pi)
bounds_u_params = [(0, 2 * np.pi)] * num_u_params

# --- Step 3: Optimize each projector P_ij(theta) against psi_initial ---
print("\nStep 3: Optimizing projector parameters P_ij* against |psi_initial>...")
qubit_pairs_indices = [(i, i + 1) for i in range(NUM_QUBITS - 1)]
optimized_projectors_P_star = []

for q_pair_idx, (q1, q2) in enumerate(qubit_pairs_indices):
    print(f"  Optimizing P_{q1}{q2}(theta) acting on qubits ({q1}, {q2})...")
    
    proj_00_full_static_spo = proj_00_on_qubits_static_pauli_sum[(q1, q2)]

    def cost_function_projector(params_theta):
        try:
            u_circuit_2q = u_ansatz_2q.assign_parameters(params_theta)
            u_full_qc = QuantumCircuit(NUM_QUBITS)
            u_full_qc.compose(u_circuit_2q, qubits=[q1, q2], inplace=True)
            u_full_op = Operator(u_full_qc)
            p_ij_full_op_object = u_full_op @ proj_00_full_static_spo @ u_full_op.adjoint()
            expectation_value = np.real(psi_initial_vector.expectation_value(p_ij_full_op_object))
            print(f"  Expectation value for P_{q1}{q2}: {expectation_value}", end='\r')
            return expectation_value
        except Exception as e_cost:
            print(f"ERROR in cost_function_projector for P_{q1}{q2} with params {params_theta}: {e_cost}.")
            return np.inf # Return a large value for errors

    # dual_annealing doesn't use x0 in the same way as local optimizers.
    # It explores based on bounds and its internal strategy.
    
    try:
        # Using dual_annealing
        da_result = dual_annealing(
            func=cost_function_projector,
            bounds=bounds_u_params,
            maxiter=DA_MAXITER,
            initial_temp=DA_INITIAL_TEMP,
            visit=DA_VISIT,
            accept=DA_ACCEPT,
            seed=SEED # For reproducibility of dual_annealing's stochastic parts
            # no_local_search=False # Default, True can speed up if local search isn't helpful
        )
        
        optimized_u_params = da_result.x
        min_expectation_value = da_result.fun
        nfev = da_result.nfev # Number of function evaluations
        success = da_result.success # Indicates if maxiter was reached (True) or other condition.
        message = da_result.message

        print(f"    P_{q1}{q2}*: Optimized <psi_init|P*|psi_init> = {min_expectation_value:.6e}, Evals = {nfev}, Success: {success}, Msg: {message}")

        if np.isinf(min_expectation_value) or np.isnan(min_expectation_value):
            print(f"    ERROR: Optimization for P_{q1}{q2}* failed (Inf/NaN result), likely due to Qiskit errors in cost function.")
            continue # Skip to the next projector

        # Construct the optimized P_ij* Operator
        u_circuit_2q_opt = u_ansatz_2q.assign_parameters(optimized_u_params)
        u_full_qc_opt = QuantumCircuit(NUM_QUBITS)
        u_full_qc_opt.compose(u_circuit_2q_opt, qubits=[q1, q2], inplace=True)
        u_full_op_opt = Operator(u_full_qc_opt)
        
        p_ij_star_op_object = u_full_op_opt @ proj_00_full_static_spo @ u_full_op_opt.adjoint()
        optimized_projectors_P_star.append({'qubits': (q1,q2), 'P_star_op': p_ij_star_op_object})

    except RuntimeError as e_scipy:
        print(f"    ERROR: SciPy Dual Annealing optimization failed for P_{q1}{q2}*: {e_scipy}")
        continue
    except Exception as e_general_opt:
        print(f"    ERROR: General error during optimization P_{q1}{q2}*: {e_general_opt}")
        continue

# --- Step 4: Iteratively apply (I - P_ij*) and renormalize ---
print("\nStep 4: Iteratively applying (I - P_ij*) projection and renormalizing...")

if not optimized_projectors_P_star:
    print("Warning: No optimized projectors P_ij* were successfully created. Cannot perform iterative projection.")
    final_psi_vector = psi_initial_vector.copy()
else:
    current_psi_vector = psi_initial_vector.copy()
    identity_op = Operator(np.eye(2**NUM_QUBITS, dtype=complex))
    projection_order = [(0,1), (1,2), (2,3), (3,4)]

    for q_idx, (q_target1, q_target2) in enumerate(projection_order):
        p_star_info = next((p_info for p_info in optimized_projectors_P_star if p_info['qubits'] == (q_target1, q_target2)), None)

        if p_star_info is None:
            print(f"  Warning: No optimized P_star found for qubits ({q_target1},{q_target2}). Skipping this projection step.")
            continue
        
        p_ij_star_op = p_star_info['P_star_op']
        print(f"  Applying (I - P_{q_target1}{q_target2}*) to current state...")

        k_ij_op = identity_op - p_ij_star_op
        projected_psi_unnormalized = current_psi_vector.evolve(k_ij_op)
        
        vec_data = projected_psi_unnormalized.data
        norm_val = np.linalg.norm(vec_data)

        if norm_val < 1e-9:
            print(f"    WARNING: State norm after applying (I - P_{q_target1}{q_target2}*) is near zero ({norm_val:.2e}).")
            print(f"    Stopping iterative projection.")
            current_psi_vector = Statevector(projected_psi_unnormalized.data, dims=projected_psi_unnormalized.dims())
            break 
        
        normalized_data_arr = projected_psi_unnormalized.data / norm_val
        current_psi_vector = Statevector(normalized_data_arr, dims=projected_psi_unnormalized.dims())
        new_norm_check = np.linalg.norm(current_psi_vector.data)
        print(f"    State renormalized. New norm after explicit creation: {new_norm_check:.7f}")

    final_psi_vector = current_psi_vector

# --- Step 5: Analyze the Final State ---
print("\nStep 5: Final Projected State Analysis...")

final_norm_val = np.linalg.norm(final_psi_vector.data)
if final_norm_val > 1e-9:
    print(f"  Normalizing final_psi_vector before analysis (current norm: {final_norm_val:.7f})")
    final_psi_vector = Statevector(final_psi_vector.data / final_norm_val, dims=final_psi_vector.dims())
    print(f"  Norm of final_psi_vector after re-check: {np.linalg.norm(final_psi_vector.data):.7f}")
else:
    print(f"  Final state is a near-zero vector (norm: {final_norm_val:.2e}). Analysis might be limited.")

print("Initial state |psi_initial> (first 4 amplitudes):")
print(psi_initial_vector.data[:4])
print("\nFinal state |psi_final> after iterative projections (first 4 amplitudes):")
print(final_psi_vector.data[:4])

try:
    if final_norm_val < 1e-9:
        print("\nFidelity calculation skipped as final state is a zero vector.")
        fidelity_final_vs_initial = 0.0
    else:
        if abs(np.linalg.norm(psi_initial_vector.data) - 1.0) > 1e-9:
             print("Warning: Initial state vector norm is not 1. Re-normalizing for fidelity.")
             psi_initial_vector = Statevector(psi_initial_vector.data / np.linalg.norm(psi_initial_vector.data))
        fidelity_final_vs_initial = state_fidelity(psi_initial_vector, final_psi_vector)
        print(f"\nFidelity(|psi_initial>, |psi_final>): {fidelity_final_vs_initial:.8f}")

    print("\nExpectation values of P_ij* with respect to |psi_initial> (should be small):")
    for p_info in optimized_projectors_P_star:
        q1_val, q2_val = p_info['qubits'] # Renamed to avoid conflict with outer q1, q2
        exp_val = np.real(psi_initial_vector.expectation_value(p_info['P_star_op']))
        print(f"  <psi_initial| P_{q1_val}{q2_val}* |psi_initial>: {exp_val:.6e}")

    print("\nExpectation values of P_ij* with respect to |psi_final> (may not be small):")
    for p_info in optimized_projectors_P_star:
        q1_val, q2_val = p_info['qubits'] # Renamed
        if final_norm_val > 1e-9:
            exp_val_final = np.real(final_psi_vector.expectation_value(p_info['P_star_op']))
            print(f"  <psi_final| P_{q1_val}{q2_val}* |psi_final>: {exp_val_final:.6e}")
        else:
            print(f"  <psi_final| P_{q1_val}{q2_val}* |psi_final>: Not computed (final state is near zero vector)")

except Exception as e_final_analysis:
    print(f"ERROR during final state analysis: {e_final_analysis}")
    if 'final_psi_vector' in locals(): # Check if final_psi_vector is defined
        print(f"DEBUG: Norm of final_psi_vector right before error: {np.linalg.norm(final_psi_vector.data)}")
        print(f"DEBUG: final_psi_vector.is_valid(): {final_psi_vector.is_valid(atol=1e-9) if hasattr(final_psi_vector, 'is_valid') else 'is_valid not available'}")

print("\n--- Iterative Projection Script Finished ---")

NumPy random seed set to: 43
--- Quantum State Iterative Projection (Using SciPy Dual Annealing) ---
Configuration: Qubits=5, Psi Depth=5
Optimizer: SciPy DualAnnealing, Maxiter=5000

Step 1: Creating initial state |psi_initial>...
|psi_initial> statevector created with 30 parameters.

Step 2: Parametrized projectors P_ij(theta) structure defined.
U(theta) for projectors is TwoLocal with 8 parameters (reps=1).

Step 3: Optimizing projector parameters P_ij* against |psi_initial>...
  Optimizing P_01(theta) acting on qubits (0, 1)...
    P_01*: Optimized <psi_init|P*|psi_init> = 3.526197e-02, Evals = 144274, Success: True, Msg: ['Maximum number of iteration reached']
  Optimizing P_12(theta) acting on qubits (1, 2)...


KeyboardInterrupt: 